# 1세대: 문장‧코드 자동완성 (First‑Gen Autocomplete)

이 노트북은 **간단한 문장/코드 자동완성** 예제를 통해 1세대 에이전트 코딩(프롬프트 → 연속 텍스트/코드 생성)의 작동 방식을 보여줍니다. 

**구성**
- OpenAI Python SDK 설치 & 키 설정
- 문장 자동완성 함수 `autocomplete_text`
- 코드 자동완성 함수 `autocomplete_code`
- (선택) 간단한 Gradio UI로 실험해보기

> Colab에서 **런타임 → 런타임 다시 시작**이 필요한 경우가 있습니다. 셀을 순서대로 실행하세요.

In [ ]:
#@title 0) 라이브러리 설치
!pip -q install --upgrade openai tiktoken gradio
print('✅ 설치 완료')


In [ ]:
#@title 1) API 키 설정 및 클라이언트 초기화
import os, getpass
from openai import OpenAI

if 'OPENAI_API_KEY' not in os.environ or not os.environ['OPENAI_API_KEY']:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('🔑 Enter your OpenAI API key: ')

client = OpenAI()

# 기본 모델은 경량/저비용 범용 텍스트 모델을 사용합니다. 필요시 다른 텍스트 모델로 교체하세요.
MODEL = 'gpt-4o-mini'  # 예: 'gpt-4o', 'gpt-4.1-mini' 등으로 변경 가능
print('✅ API 준비 완료, MODEL =', MODEL)


In [ ]:
#@title 2) 문장 자동완성 함수
from typing import Optional

def autocomplete_text(prefix: str,
                      n_tokens: int = 96,
                      temperature: float = 0.7,
                      system_prompt: Optional[str] = None) -> str:
    """사용자가 제공한 문장/단락 prefix를 자연스럽게 이어서 작성합니다."""
    if system_prompt is None:
        system_prompt = (
            'You are an autocomplete engine. Continue the user\'s text naturally '
            'without adding meta commentary or lists unless the style requires it.'
        )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prefix}
    ]
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=n_tokens,
        temperature=temperature,
    )
    return resp.choices[0].message.content.strip()

demo_prefix = "어젯밤 나는 예상치 못한 이메일을 받았다. 발신자는 이름도 밝히지 않은 채 단 한 문장만 남겼다:"
print('입력(prefix):\n', demo_prefix)
print('\n--- 자동완성 결과 ---\n')
print(autocomplete_text(demo_prefix))


In [ ]:
#@title 3) 코드 자동완성 함수 (Python/JS 등)
LANG_SNIPPET_GUIDE = {
    'python': (
        'Continue the following Python code. Keep style consistent, avoid extra comments unless helpful.'
    ),
    'javascript': (
        'Continue the following JavaScript code. Keep style consistent, avoid extra comments unless helpful.'
    ),
    'java': 'Continue the following Java code.',
    'cpp': 'Continue the following C++ code.'
}

def autocomplete_code(language: str,
                      code_prefix: str,
                      n_tokens: int = 128,
                      temperature: float = 0.4) -> str:
    """주어진 언어와 코드 prefix를 이어서 작성합니다."""
    guide = LANG_SNIPPET_GUIDE.get(language.lower(), 'Continue the following code.')
    system_prompt = (
        'You are a code autocomplete engine. Continue the code faithfully. '
        'Do not wrap in markdown fences unless they are already present.'
    )
    user_content = f"Language: {language}\nInstruction: {guide}\n\n{code_prefix}"
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content}
        ],
        max_tokens=n_tokens,
        temperature=temperature,
    )
    return resp.choices[0].message.content

# Python 예시 prefix
py_prefix = (
    "def fibonacci(n: int):\n"
    "    \"\"\"Return the n-th Fibonacci number using iterative DP.\"\"\"\n"
    "    if n < 0:\n"
    "        raise ValueError('n must be >= 0')\n"
    "    a, b = 0, 1\n"
    "    for _ in range(n):\n"
    "        a, b = b, a + b\n"
    "    return a\n\n"
    "# 간단한 벤치마크\n"
    "if __name__ == '__main__':\n"
    "    import time\n"
    "    start = time.time()\n"
    "    # TODO: 여러 입력에 대해 실행하고 결과를 출력해보자\n"
)
print('--- 코드 자동완성 (Python) ---')
print(autocomplete_code('python', py_prefix))

# JavaScript 예시 prefix
js_prefix = (
    "function debounce(fn, delay) {\n"
    "  let timer = null;\n"
    "  return function(...args) {\n"
    "    clearTimeout(timer);\n"
    "    timer = setTimeout(() => fn.apply(this, args), delay);\n"
    "  };\n"
    "}\n\n"
    "// TODO: 아래 search 함수를 debounce로 감싸서 입력 이벤트 최적화\n"
)
print('\n--- 코드 자동완성 (JavaScript) ---')
print(autocomplete_code('javascript', js_prefix))


## 4) (선택) Gradio 미니 UI
브라우저에서 간편하게 문장/코드 자동완성을 실험할 수 있는 간단한 UI를 제공합니다.

In [ ]:
#@title 4) Gradio 앱 실행 (선택)
import gradio as gr

def ui_text(prefix, n_tokens, temperature):
    return autocomplete_text(prefix, n_tokens=n_tokens, temperature=temperature)

def ui_code(language, code_prefix, n_tokens, temperature):
    return autocomplete_code(language, code_prefix, n_tokens=n_tokens, temperature=temperature)

with gr.Blocks() as demo:
    gr.Markdown('# First‑Gen Autocomplete Demo')
    with gr.Tab('Text Autocomplete'):
        prefix = gr.Textbox(label='Text prefix', value='오늘 아침에 나는 평소보다 일찍 눈을 떴다. 창밖을 보니,')
        n_tokens = gr.Slider(16, 512, value=96, step=1, label='max_tokens')
        temperature = gr.Slider(0.0, 1.5, value=0.7, step=0.1, label='temperature')
        out = gr.Textbox(label='Completion', lines=8)
        btn = gr.Button('Generate')
        btn.click(ui_text, inputs=[prefix, n_tokens, temperature], outputs=[out])
    with gr.Tab('Code Autocomplete'):
        language = gr.Dropdown(['python','javascript','java','cpp'], value='python', label='Language')
        code_prefix = gr.Code(label='Code prefix', value='def add(a, b):\n    # TODO: implement\n    ')
        n_tokens_c = gr.Slider(16, 512, value=128, step=1, label='max_tokens')
        temperature_c = gr.Slider(0.0, 1.5, value=0.4, step=0.1, label='temperature')
        out_c = gr.Code(label='Completion')
        btn_c = gr.Button('Generate')
        btn_c.click(ui_code, inputs=[language, code_prefix, n_tokens_c, temperature_c], outputs=[out_c])

demo.launch()


### 참고
- Colab에서 **런타임 유형**은 Python 3(기본)으로 진행하세요. GPU는 필수 아님.
- 네트워크/키 문제로 실패하는 경우, 다시 실행하거나 키/모델명을 확인하세요.
- 자동완성 특성상 **보안 민감 정보**를 포함하지 않도록 주의하세요.